# Imports of modules

Below we import some modules necessary to make this code work properly.

You can add other modules here which you might need for your algorithm.

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import random
from scipy.spatial import distance
import time
from tqdm.notebook import tqdm
import itertools

On a utilisé le module time pour mesurer les temps d'exécution des différentes parties du code pour comprendre l'influence des paramètres sur les temps d'exécution. 

# Definitions of constants

In the cell below we define some constants that we need in the meta-heuristic.

The paths to the data files are also specified here.

In [10]:
# ILS parameters

#MAX_ITER = 10     # maximum number of iterations (Stopping criterion of ILS)
#MAX_ITER_NI = 20   # number of iterations without improvement of the objective function (Stopping criterion of ILS)
#MAX_ITER_LS = 30  # maximum number of iterations of the local search operator (Outer loop)
#MAX_SWAPS = 1      # maximum number of swaps of the local search operator (Inner loop)
#NB_SWAPS = 5       # number of swaps in the perturbation operator
#ALPHA = 0.05
optimal_sol = 2.484863919811e+07
# Path to data file

SINPUT_DATA = "InputDataEnergySmallInstance.xlsx"  # Small instance
INPUT_DATA = "InputDataEnergyLargeInstance.xlsx"  # Large instance

Les paramètres de ILS ne sont plus **définis de manière globale** mais de **manière locale** pour pouvoir les faire varier.

# Functions to load and prepare the data

In the cell below, you find three functions:
- first, read_excel_data, which returns the values of a specific sheet in an Excel file,
- then, a function to calculate the length of an eadge A--B in the network based on the coordinates of nodes A and B,
- and finally, create_data_model which fills the data dictionary with the data from the various sheets of the Excel file, as well as some dependent data, which is calculated from the raw data.

In [11]:
def read_excel_data(filename: str, sheet_name: str) -> np.ndarray:
    """Read data from an excel spreadsheet.

    :param filename:
    :param sheet_name:
    :return: sheet data
    """
    data = pd.read_excel(filename, sheet_name=sheet_name, header=None)
    values = data.values
    return values


def compute_edge_lengths(nodes_coord: np.ndarray) -> np.ndarray:
    """Compute distance matrix between each pair of nodes.

    :param nodes_coord: coordinates of each node as a N*2 array
    :return:
    """
    edges_length = np.zeros((len(nodes_coord), len(nodes_coord)))
    for i in range(0, len(nodes_coord)):
        for j in range(0, len(nodes_coord)):
            edges_length[i, j] = distance.euclidean((nodes_coord[i]), (nodes_coord[j]))
    return edges_length


def create_data_model() -> dict:
    """Create the data model which gathers all problem parameters.

    :return: data model structure
    """
    data = {}

    # This section contains all required data #
    data['SourceNum'] = read_excel_data(INPUT_DATA, "SourceNum")[0][0]

    # Nodes Cordinate Read From Excel#
    data['NodesCord'] = read_excel_data(INPUT_DATA, "NodesCord")

    # Building Cordinate Read From Excel#
    #EdgesDemand = read_excel_data(INPUT_DATA, "EdgesDemand")

    # Fixed Unit Cost
    data['FixedUnitCost'] = read_excel_data(INPUT_DATA, "FixedUnitCost")

    # Edges Length Calculation based the Nodes Cordinate
    data['EdgesLength'] = np.matrix.round(compute_edge_lengths(data['NodesCord']))

    # Fixed Instalation cost
    data['cfix'] = data['EdgesLength'] * data['FixedUnitCost']

    # Number of Nodes
    data['node_num'] = len(data['NodesCord'])

    # Revenue of Fulfiling the Edges Demand
    data['crev'] = read_excel_data(INPUT_DATA, "crev(cijrev)")

    # Penalty of Unmet Demand
    data['pumd'] = read_excel_data(INPUT_DATA, "pumd(pijumd)")

    # Cost of Heat Production in the Sources
    data['cheat'] = read_excel_data(INPUT_DATA, "cheat(ciheat)")

    # Variable Cost of Heat Transferring through the Edges
    data['cvar'] = read_excel_data(INPUT_DATA, "cvar(cijvar)")

    # Variable Thermal Losses
    data['vvar'] = read_excel_data(INPUT_DATA, "vvar(thetaijvar)")

    # Fixed Thermal Losses
    data['vfix'] = read_excel_data(INPUT_DATA, "vfix(thetaijfix)")

    # Full Load Hours for the Sources
    data['Tflh'] = read_excel_data(INPUT_DATA, "Tflh(Tiflh)")

    # Concurrence Effect
    data['Betta'] = read_excel_data(INPUT_DATA, "Betta")

    # Connect Quota
    data['Lambda'] = read_excel_data(INPUT_DATA, "Lambda")

    # Annuity Factor for Investment Cost
    data['Alpha'] = read_excel_data(INPUT_DATA, "Alpha")

    # Edges Peak Demand
    data['EdgesDemandPeak'] = read_excel_data(INPUT_DATA, "EdgesDemandPeak(dij)")

    # Edges Annual Demand
    data['EdgesDemandAnnual'] = read_excel_data(INPUT_DATA, "EdgesDemandAnnual(Dij)")

    # Edges Maximum Capacity
    data['Cmax'] = read_excel_data(INPUT_DATA, "Cmax(cijmax)")

    # Cost of Maintenance
    data['com'] = read_excel_data(INPUT_DATA, "com(cijom)")

    # Source Maximum Capacity
    data['SourceMaxCap'] = read_excel_data(INPUT_DATA, "SourceMaxCap(Qimax)")

    # Dependent Parameters
    data['kfix'] = data['cfix'] * data['Alpha'] * data['EdgesLength'] + data['com'] * data['EdgesLength']
    data['kvar'] = data['cvar'] * data['EdgesLength'] * data['Alpha']
    data['rheat'] = data['crev'] * data['EdgesDemandAnnual'] * data['Lambda']
    data['kheat'] = (data['Tflh'] * data['cheat']) / data['Betta']
    data['Etta'] = 1 - data['EdgesLength'] * data['vvar']
    data['Delta'] = data['EdgesDemandPeak'] * data['Betta'] * data['Lambda'] + data['EdgesLength'] * data['vfix']

    return data

# Functions to calculate the objective function from the solution representation

The cell below contains 2 functions to calculate the objective function of an individual:
- first `prufer_to_tree` which transforms the Prüfer representation of a solution into a tree,
- second, `tree_to_prufer` which transforms a tree into a Prüfer representation,
- third, `compute_of` which calculates the fitness (or objective function) of an individual (or a solution).

In [12]:
def prufer_to_tree(prufer_sequence: list) -> nx.Graph:
    """Transform the Prüfer representation into a tree.

    :param a: Prüfer sequence representing a tree
    :return: tree as a graph structure
    """
    return nx.from_prufer_sequence(prufer_sequence)


def tree_to_prufer(tree: nx.Graph) -> list:
    """Transform a tree into a Prüfer sequence.

    :param tree:
    :return: Prüfer sequence representing the tree

    """

    return nx.to_prufer_sequence(tree)


def compute_of(individual: list, data: dict) -> float:
    """Calculate the objective function of the individual.

    :param individual: solution as a tree in Prüfer representation
    :param data: data model including all parameters of the problem
    :return: value of the objective function for given individual
    """
    graph = prufer_to_tree(individual)
    tree_edges = list(graph.edges)
    all_pairs_path = dict(nx.all_pairs_shortest_path(graph))
    path_from_source = all_pairs_path[data['SourceNum']]

    P_in = np.zeros((len(graph.edges)+1, len(graph.edges)+1))
    P_out = np.zeros((len(graph.edges)+1, len(graph.edges)+1))

    hubs = np.unique(individual)
    spokes = list(set([i for i in range(len(individual)+2)]) - set(hubs))

    for i in spokes:
        A = path_from_source[i]
        e = 0
        for k in range(0, len(A)-1):
            if e == 0:
                P_out[A[len(A)-k-2], A[len(A)-k-1]] = 0
                e = e + 1
            else:
                P_out[A[len(A)-k-2], A[len(A)-k-1]] = sum(P_in[A[len(A)-k-1]])

            P_in[A[len(A)-k-2], A[len(A)-k-1]] = (P_out[A[len(A)-k-2], A[len(A)-k-1]] + data['Delta'][A[len(A)-k-2], A[len(A)-k-1]])/data['Etta'][A[len(A)-k-2], A[len(A)-k-1]]

    fitness = 0
    metDemand = 0
    for i in range(len(tree_edges)):
        fitness = fitness + data['kfix'][tree_edges[i]] - data['rheat'][tree_edges[i]] + data['kvar'][tree_edges[i]] * (P_in[tree_edges[i][0], tree_edges[i][1]])
        metDemand = metDemand + 2 * data['EdgesDemandAnnual'][tree_edges[i]] * data['pumd'][tree_edges[i]]
    fitness = fitness + data['kheat'][data['SourceNum']] * sum(P_in[data['SourceNum']])
    fitness = fitness + 0.5*((data['EdgesDemandAnnual'] * data['pumd']).sum() - metDemand)

    return fitness

Nous utilisons la représentation de **Prüfer des arbres**. La représentation de Prüfer permet de créer une **bijection entre un arbre et une liste.** Cela signifie qu'au lieu de modifier directement l'arbre, on peut modifier la liste de Prüfer, puis inverser la bijection pour retrouver l'arbre avec les modifications appliquées.

L'avantage de cette représentation réside dans la **rapidité d'exécution des modifications d'un arbre**. Elle simplifie également le code, car un arbre est une **structure naturellement récursive**, ce qui aurait nécessité que toutes nos **fonctions soient récursives pour interagir avec l'arbre**. En revanche, la liste de Prüfer est une **structure linéaire**, ce qui permet une interaction plus facile.

# Functions to create solutions or individuals

The cell below contains two functions regarding individuals:

- first, `generate_individual` to create a random individual,
- second, `initial_solution` which returns this single randomly generated individual and its fitness.

In [13]:
def generate_individual(nb_nodes: int) -> list:
    """Generate a random individual.

    The created nodes will be labelled as integer from 0 to `nb_nodes`-1

    :param nb_nodes: total number of nodes in an individual tree
    :return: Prüfer representation of a random tree with given number of nodes
    """
    individual = np.ndarray.tolist(np.random.randint(0, high=nb_nodes-1, size=nb_nodes-2, dtype='int'))

    return individual


def initial_solution(data: dict) -> tuple:
    """Generate a random solution and calculate its fitness.

    :param data: data model including all parameters of the problem
    :return: random solution in Prüfer representation, and its objective function value
    """
    # here we are generating only one initial solution
    solution = generate_individual(data['node_num'])
    return solution, compute_of(solution, data)

# Functions for the local search

Below you can find functions to perform a local search:
- first the general high-level `local_search` function,
- second the `swap` function, which implements a swap operator,
- third the `swap_neighborhood` function which generates the neighborhood based on the swap operator,
- and finally the `unique_pairs` function, used by `swap_neighborhood`, wich generates unique pairs indexes.

In [14]:
def local_search(sol: list, of: float, data: dict , MAX_ITER_LS : int) -> tuple:
    """Perform a local search.

    :param sol: input solution
    :param of: objective function value of input solution
    :param data: data model including all parameters of the problem
    :return: best neighbouring solution, and its objective function value
    """
    
    best_of = of
    best_sol = sol

    # Main loop local search
    # Local search operators is repeated MAX_ITER_LS times
    for _ in range(MAX_ITER_LS):

        #print('MAX_ITER_LS',nb_iterations)
        # use an operator to perform local search
        best_of , best_sol = swap_neighborhood(best_sol, best_of, data)
    return best_sol, best_of 


def swap(i: int, j: int, sol: list) -> list:
    """Swap operator.

    This function simply swaps the nodes at position `i` and `j` in the Prüfer sequence.

    :param i: first position in `sol` sequence
    :param j: second position in `sol` sequence
    :param sol: parent solution to modify
    :return: new solution
    """
    
    sol[i], sol[j] = sol[j], sol[i]

    return sol


# The following function is a function to generate the neighbours of the given solution "sol"
def swap_neighborhood(sol: list, best_of: float, data: dict) -> tuple:
    """Neighborhood generation with a swap operator.

    All pairs of possible neighbours are investigated and create a solution and its objective function value each.

    :param sol: parent solution which neighbourhood to search
    :param best_of: parent solution objective function value
    :param data: data model including all parameters of the problem
    :return: list of neighbouring solutions, and corresponding list of neighbours objective function values
    """
    n = data['node_num']-2
    best_sol= sol
    computed = 0
    for i, j in unique_pairs(n):
        sol = swap(i,j,sol)
        computed = compute_of(sol,data)
        if computed< best_of:
            best_sol = sol
            best_of = computed
    return best_of , best_sol
    

def unique_pairs(n: int):
    """Generator producing pairs of indexes in range(n).

    :param n:
    :return: pair of indexes (generator)
    """
    for i in range(n-1):
        for j in range(i + 1, n-1):
            yield i, j

J'ai changé la logique du code pour l'optimiser, en effet je cherche le minimum des voisins directement dans la fonction swap_neighborhood pour éviter de devoir trouver le minimum d'un liste dans local_search.

# Functions for the perturbation of solutions

In [15]:
def random_swap(sol: list, data : dict) -> list:
    """Random perturation.

    Applies `NB_SWAPS` random swap operations on the solution.

    :param sol: parent solution
    :return: modified solution
    """
    n = data['node_num']-2
    i = random.randint(0, n - 1)
    j = random.randint(0, n - 1)
    while (i==j):
        j = random.randint(0, n - 1)
    sol = swap(i, j, sol)

    return sol

# This function is the main body of the perturbation operator, wherein the random_swap function is called
def perturb(sol: list, data: dict, NB_SWAPS : int) -> tuple:
    """Add random perturbations on the solution.

    :param sol: current solution to modify
    :param data: data model including all parameters of the problem
    :return: new solution, and its objective function value
    """
    pert_sol=sol
    for i in range(NB_SWAPS):
        pert_sol=random_swap(pert_sol, data)
    pert_of = compute_of(sol,data)
    
    return pert_sol, pert_of

# Analyse du temps d'exécution des différentes parties #

Un tour de boucle du local search ~ 0.7 s. 

un tour de boucle de perturb peut être considéré en temps constant.

Dans notre fonction main, on fait max(MAX_ITER_NI , MAX_ITER) * MAX_ITER_LS la boucle du local search 

Avec les paramètres proposés MAX_ITER = 100, MAX_ITER_NI = 50 et MAX_ITER_LS = 100 cela prendrait 116 minutes ...


# Main


In [16]:
def main(MAX_ITER : int, MAX_ITER_NI : int , MAX_ITER_LS : int , MAX_SWAPS : int , NB_SWAPS: int , ALPHA : float) -> float:
    
    #start=time.time()
    
    
    #Parameter
    #MAX_ITER      # maximum number of iterations (Stopping criterion of ILS)
    #MAX_ITER_NI    # number of iterations without improvement of the objective function (Stopping criterion of ILS)
    #MAX_ITER_LS   # maximum number of iterations of the local search operator (Outer loop)
    #MAX_SWAPS       # maximum number of swaps of the local search operator (Inner loop)
    #NB_SWAPS        # number of swaps in the perturbation operator
    #ALPHA          # critère d'acceptation sur la pertubation

    # *************************Initialisation***************************
    # initialise the data
    
    data = create_data_model()

    # init number of iterations
    nb_iterations = 0

    # find initial solution (just one) with a constructive heuristic
    best_sol, best_of = initial_solution(data)

    # **************************Local Search******************************

    best_sol, best_of = local_search(best_sol, best_of, data, MAX_ITER_LS)
    best_known = best_sol
    best_of_known = best_of

    # ********************************************************************

    # ******************************ILS***********************************
    flag_continue = True
    improve = 0

    while flag_continue and nb_iterations <= MAX_ITER and improve <= MAX_ITER_NI:

        nb_iterations += 1
        
        
        # ******************Perturbation**********************************
        pert_sol, pert_of = perturb(best_sol, data, NB_SWAPS)
        # print(pert_of)
        #print('\nPertubation DONE')
        # ******************Local Search***********************************
        best_sol_pert, best_of_pert = local_search(pert_sol, pert_of, data, MAX_ITER_LS)
        # print(best_of_pert)
        #print('\nLocal Search DONE')
        # ******************Aceptance criterion***************************
        if(best_of_pert < best_of_known):
            best_known = best_sol_pert
            best_of_known = best_of_pert
            improve = 0
        else:
            improve += 1

        if (best_of_pert < best_of * (1 + ALPHA)):
            best_of = best_of_pert
            best_sol = best_sol_pert
        #else:
            #flag_continue = False
            #print('Stop à cause du flag')
        print('Nombre d\'itération :',nb_iterations, end='\r')
    return float(best_of_known)

In [17]:
resultat = main(2,1,10,1,1,0.05)
print(f'On a une fonciton obejctive {round(resultat):e} avec un écart de la valeur optimal de {round(abs(resultat- optimal_sol)):e}')

On a une fonciton obejctive 7.695123e+08 avec un écart de la valeur optimal de 7.446637e+08


Après 40 minutes d'exécution, on obtient un résultat vraiment **peu satisfaisant**. Cela peut être dû à un mauvais choix des paramètres. En effet, soit notre algorithme de recherche locale (ILS) ne cherche pas assez en profondeur les minima locaux (**paramètre MAX_ITER_LS trop faible**), soit il n'explore pas assez l'espace de recherche pour découvrir de nouvelles zones avec des minima locaux beaucoup plus faibles (**paramètres ALPHA et NB_SWAPS trop faibles**). On peut également remettre en question le choix des opérateurs de perturbation utilisés dans la recherche locale et dans la perturbation de la solution, qui pourraient ne pas être assez performants.

# Optimisation naïf des metaparamètres #

On a vu que notre fonction main dépendait des **metaparamètres** suivants :<br>

    MAX_ITER    # maximum number of iterations (Stopping criterion of ILS)
    MAX_ITER_NI    # number of iterations without improvement of the objective function (Stopping criterion of ILS)
    MAX_ITER_LS   # maximum number of iterations of the local search operator (Outer loop)
    MAX_SWAPS       # maximum number of swaps of the local search operator (Inner loop)
    NB_SWAPS        # number of swaps in the perturbation operator
    ALPHA          # critère d'acceptation sur la pertubation

La solution retournée par notre fonction principale dépend directement de ses paramètres. Nous allons utiliser différents algorithmes pour essayer de trouver un jeu de paramètres optimaux permettant d'obtenir la meilleure solution.

### Algorithme  grid search ###

In [80]:
# Définir les valeurs des hyperparamètres
param_grid = {
    'MAX_ITER': [5],
    'MAX_ITER_NI': [1, 10, 20],
    'MAX_ITER_LS': [1, 10, 20],
    'MAX_SWAPS': [5],
    'NB_SWAPS': [1, 5, 10],
    'ALPHA': [0.1]
}

# Créer toutes les combinaisons possibles
keys, values = zip(*param_grid.items())
combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

def evaluate_algorithm(params):
    return main(**params)

best_score = float('+inf')
best_params = None

# Utiliser tqdm pour suivre la progression
for combination in tqdm(combinations, desc="Grid Search Progress"):
    score = evaluate_algorithm(combination)
    if score < best_score:
        best_score = score
        best_params = combination

print("Best hyperparameters:", best_params)
print("Best score:", best_score)


Grid Search Progress:   0%|          | 0/27 [00:00<?, ?it/s]

Best hyperparameters: {'MAX_ITER': 5, 'MAX_ITER_NI': 1, 'MAX_ITER_LS': 20, 'MAX_SWAPS': 5, 'NB_SWAPS': 10, 'ALPHA': 0.1}
Best score: 894289613


Finalement, l'algorithme grid search n'est pas adapté ici, car chaque itération prend environ 30 secondes avec de petits paramètres et jusqu'à plusieurs minutes avec de gros paramètres. Avec une grille de seulement **3 niveaux** et de 6 dimensions, nous aurions **729 itérations** de notre fonction principale.

En utilisant la bibliothèque Tqdm, nous pouvons suivre la progression de l'algorithme. Il est estimé que l'algorithme prendrait **5 heures** pour se terminer, ce qui n'est pas envisageable.

Il faut trouver une méthode d'optimisation **qui limite le nombre d'appels de notre fonction main**.

Une première solution consiste à effectuer une recherche en grille sur **un nombre restreint de paramètres**, ce qui réduirait la dimension de notre grille et, par conséquent, le nombre total d'appels. Cependant, cette approche pourrait nous faire manquer certaines **synergies entre les paramètres testés et non testés**.

Une solution autre pour contourner ce problème est d'utiliser une méthode d'optimisation probabiliste, telle que **l'optimisation bayésienne**, pour limiter le nombre d'appels à la fonction main.

Nous allons maintenant observer l'impact des metaparamètres sur la solution trouvée.

# Impact des metaparamètres sur la solution #

Dans cette partie, nous allons essayer de comprendre l'impact des différents métaparamètres sur la fonction objectif.

En réalité, nous avons deux paramètres à optimiser pour notre fonction : la fonction objectif, qui nous renseigne sur la qualité de la solution trouvée, mais également le temps d'exécution. En effet, si le temps d'exécution n'était pas à prendre en compte, on pourrait se contenter de faire une recherche du maximum global grâce à des méthodes exactes. Ici, on prend en compte le temps et donc il faut essayer de trouver une combinaison de paramètres qui donne une bonne solution en un temps raisonnable.

Une première observation est que les paramètres MAX_ITER, MAX_ITER_NI et MAX_ITER_LS sont proportionnels au temps d'exécution ainsi qu'à la qualité de la solution.

En effet, plus on augmente ces paramètres, plus la solution s'améliore, mais plus l'exécution prend du temps. On en conclut qu'il n'est donc pas nécessaire de trouver une valeur optimale pour ces paramètres. Le choix de ces paramètres se résume au temps d'exécution que l'on veut bien allouer à notre projet.

Le lien entre les paramètres ALPHA, MAX_SWAPS, NB_SWAPS et la fonction objectif n'est pas clair et dépend de la solution initiale. En effet, si notre solution initiale est loin du minimum global, il faudra beaucoup explorer et donc avoir les paramètres d'exploration ALPHA, MAX_SWAPS, NB_SWAPS élevés. À l'inverse, si notre solution initiale est proche du minimum global, des paramètres d'exploration trop élevés pourraient nous faire rater ce minimum.

De plus, sauf erreur de notre part, le paramètre MAX_SWAPS ne sert à rien dans notre implémentation.

Nous nous retrouvons donc avec deux paramètres à tester via l'algorithme de grid search.

De plus on stocke les valeurs de la fonction objectif pour les analyser par la suite.


In [18]:
# Dictionnaire pour stocker les valeurs de la fonction objectif
objective_values = {}

### Algorithme Grid Search ###

In [19]:
param_grid = {
    'MAX_ITER': [13],
    'MAX_ITER_NI': [9],
    'MAX_ITER_LS': [10],
    'MAX_SWAPS': [1],
    'NB_SWAPS': [1,10,50],
    'ALPHA': [0.01,0.1,0.2,0.3]
}

# Créer toutes les combinaisons possibles
keys, values = zip(*param_grid.items())
combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

# Fonction pour évaluer l'algorithme
def evaluate_algorithm(params):
    return main(**params)

# Initialiser les meilleures scores et paramètres
best_score = float('+inf')
best_params = None


# Utiliser tqdm pour suivre la progression
for combination in tqdm(combinations, desc="Grid Search Progress"):
    score = evaluate_algorithm(combination)
    
    # Extraire les valeurs de MAX_SWAPS et ALPHA
    nb_swaps = combination['NB_SWAPS']
    alpha = combination['ALPHA']
    # Stocker les valeurs dans le dictionnaire
    if (nb_swaps, alpha) not in objective_values:
        objective_values[(nb_swaps, alpha)] = []
        objective_values[(nb_swaps, alpha)].append(score)
    else : 
        objective_values[(nb_swaps, alpha)].append(score)
    # Mettre à jour le meilleur score et les meilleurs paramètres
    if score < best_score:
        best_score = score
        best_params = combination

print("Best hyperparameters:", best_params)
print("Best score:", best_score)

Grid Search Progress:   0%|          | 0/12 [00:00<?, ?it/s]

Best hyperparameters: {'MAX_ITER': 13, 'MAX_ITER_NI': 9, 'MAX_ITER_LS': 10, 'MAX_SWAPS': 1, 'NB_SWAPS': 50, 'ALPHA': 0.3}
Best score: 714886304.8297217


In [20]:
# Afficher les valeurs de la fonction objectif
for key, values in objective_values.items():
    print(f"MAX_SWAPS: {key[0]}, ALPHA: {key[1]}, Objective Values: {values[0]:e}")

MAX_SWAPS: 1, ALPHA: 0.01, Objective Values: 8.470660e+08
MAX_SWAPS: 1, ALPHA: 0.1, Objective Values: 8.552498e+08
MAX_SWAPS: 1, ALPHA: 0.2, Objective Values: 8.957163e+08
MAX_SWAPS: 1, ALPHA: 0.3, Objective Values: 8.174403e+08
MAX_SWAPS: 10, ALPHA: 0.01, Objective Values: 7.781913e+08
MAX_SWAPS: 10, ALPHA: 0.1, Objective Values: 8.239375e+08
MAX_SWAPS: 10, ALPHA: 0.2, Objective Values: 7.873321e+08
MAX_SWAPS: 10, ALPHA: 0.3, Objective Values: 8.019411e+08
MAX_SWAPS: 50, ALPHA: 0.01, Objective Values: 7.426026e+08
MAX_SWAPS: 50, ALPHA: 0.1, Objective Values: 7.779779e+08
MAX_SWAPS: 50, ALPHA: 0.2, Objective Values: 8.044435e+08
MAX_SWAPS: 50, ALPHA: 0.3, Objective Values: 7.148863e+08


Finalement on n'arrive pas à trouver de tendance claire et on a plus l'impression que la fonction objectif dépend beaucoup de la solution initiale générée aléatoirement. Cela peut être du au fait qu'on a choisi un nombre relativement faible pour MAX_ITER et pour MAX_ITER_LS afin de limiter le temps d'éxecution.

# Conclusion #

Pour conclure, notre algorithme ILS est plûtot décevant en l'état actuel car il se trompe d'une puissance de 10 par rapport à la solution optimale. Cela peut être dû à divers facteurs comme le mauvais choix des opérateurs de pertubation utilisés dans la fonction local search ou dans la fonction de pertubation de la solution.

Les perspectives d'amélioration sont les suivantes:

- Essayer d'autres opérateurs de pertubation et les comparer entre eux
- Essayer d'implémenter une optimisation bayésienne afin d'optimiser les paramètres ALPHA et NB_SWAPS
- Disposer de serveurs pouvant faire tourner la fonction main pendant plusieurs heures avec les paramètres et les opérateurs optimaux afin d'arriver à une solution concluante

